In [4]:
import sys

In [5]:
import scipy.signal as ss
from scipy import signal
import h5py
import neo
import nixio
import numpy as np
import matplotlib.pyplot as plt
import json
from scipy.interpolate import interp2d
from scipy.stats import linregress
import os
from scipy import io
import quantities as pq
from collections import Counter

In [6]:
nchan = 51                    # number of channels on electrode
ntime = 1000                  # [ms]
trial_dur = 1000              # [ms]
nstim = 10                    # number of trials all stimuli
ntrials = 5                   # number of trials each stimulus type
nstim_types = 2
time_stim_onset = 500         # [ms]
time_stim_dur = 250           # [ms] duration of stimulus
ncells = 21                   # number of cell types
stim_string = ['White Flash', 'Black Flash']
stim_type = 0                 # white (0) or black (1) flash
layer_borders = [-0, -100, -310, -430, -650, -850] # [1E-6*m]
total_time = 10500            # ms

In [7]:
sim_data_dir = '/Users/atleeskelandrimehaug/Dropbox (Simula)/scripts_for_bmtk/data_files/sim'

In [8]:
cell_names = ['e23Cux2',
 'e4Nr5a1',
 'e4Rorb',
 'e4Scnn1a',
 'e4other',
 'e5Rbp4',
 'e5noRbp4',
 'e6Ntsr1',
 'i1Htr3a',
 'i23Htr3a',
 'i23Pvalb',
 'i23Sst',
 'i4Htr3a',
 'i4Pvalb',
 'i4Sst',
 'i5Htr3a',
 'i5Pvalb',
 'i5Sst',
 'i6Htr3a',
 'i6Pvalb',
 'i6Sst']
layer_names = ['e23', 'e4', 'e5', 'e6', 'i1', 'i23', 'i4', 'i5', 'i6']

In [10]:
f_rates_trial_avg = np.zeros((ncells,nstim_types,ntime))

fir_rates_all_trials = np.zeros((ncells,nstim_types,ntrials,ntime))

for cell_nr, cell_name in enumerate(cell_names):
    filename = 'mod_21_jan_20/spikes_sorted_by_pop/onlySpikeTimesPerPopulation/'+\
        'onlySpikeTimes_%s.txt' % cell_name
    total_time = 10500
    
    print(cell_nr, cell_name)
    loadpath = os.path.join(sim_data_dir, filename)
    
    a = open(loadpath, 'r')
    
    spike_times = np.asarray(a.readlines(), dtype = float)

    t = np.linspace(0, total_time, total_time+1)

    entries = np.digitize(spike_times, t)
    
    count = Counter(entries)
    entries_values = list(count.keys())
    spike_time_freqs = list(count.values())
    
    #print('entries_values: ', entries_values)
    #print('entries_values: ', spike_time_freqs)
    
    firing_rate = np.zeros_like(t)

    firing_rate[entries_values] = spike_time_freqs
    
    firing_rate_reshaped = np.zeros((nstim, trial_dur))

    for i in range(nstim):
        firing_rate_reshaped[i] = firing_rate[i*1000:(i+1)*1000]
    
    for itrial in range(ntrials):
        fir_rates_all_trials[cell_nr,0,itrial] = firing_rate_reshaped[2*itrial]
        
        fir_rates_all_trials[cell_nr,1,itrial] = firing_rate_reshaped[2*itrial+1]

    # Calculate trial averages
    f_rates_trial_avg[cell_nr, 0:1] = np.mean(firing_rate_reshaped[0::2],0)
    f_rates_trial_avg[cell_nr, 1:2] = np.mean(firing_rate_reshaped[1::2],0)

0 e23Cux2
1 e4Nr5a1
2 e4Rorb
3 e4Scnn1a
4 e4other
5 e5Rbp4
6 e5noRbp4
7 e6Ntsr1
8 i1Htr3a
9 i23Htr3a
10 i23Pvalb
11 i23Sst
12 i4Htr3a
13 i4Pvalb
14 i4Sst
15 i5Htr3a
16 i5Pvalb
17 i5Sst
18 i6Htr3a
19 i6Pvalb
20 i6Sst


#### Number of cells

In [11]:
#num_neurons = np.zeros(ncells)
num_neurons = np.zeros(len(cell_names))
for cell_nr, cell_name in enumerate(cell_names):
    filename = 'mod_21_jan_20/spikes_sorted_by_pop/neuronIdsPerPopulation/'+\
                '%s.txt' % cell_name
    
    print(cell_nr, cell_name)
    
    fullpath = os.path.join(sim_data_dir, filename)
    
    a = open(fullpath, 'r')
    
    neuron_ids = np.asarray(a.readlines(), dtype = int)
    
    num_neurons[cell_nr] = len(neuron_ids)
    print(len(neuron_ids))

0 e23Cux2
12689
1 e4Nr5a1
1248
2 e4Rorb
2605
3 e4Scnn1a
3072
4 e4other
3329
5 e5Rbp4
6010
6 e5noRbp4
1559
7 e6Ntsr1
12882
8 i1Htr3a
999
9 i23Htr3a
1107
10 i23Pvalb
640
11 i23Sst
464
12 i4Htr3a
270
13 i4Pvalb
963
14 i4Sst
553
15 i5Htr3a
117
16 i5Pvalb
613
17 i5Sst
555
18 i6Htr3a
192
19 i6Pvalb
1052
20 i6Sst
1059


In [18]:
num_neurons_layers = {}
num_neurons_layers['23e'] = num_neurons[0].tolist()
num_neurons_layers['4e'] = np.sum(num_neurons[1:5]).tolist()
num_neurons_layers['5e'] = np.sum(num_neurons[5:7]).tolist()
num_neurons_layers['6e'] = num_neurons[7].tolist()
num_neurons_layers['1i'] = num_neurons[8].tolist()
num_neurons_layers['23i'] = np.sum(num_neurons[9:12]).tolist()
num_neurons_layers['4i'] = np.sum(num_neurons[12:15]).tolist()
num_neurons_layers['5i'] = np.sum(num_neurons[15:18]).tolist()
num_neurons_layers['6i'] = np.sum(num_neurons[18:]).tolist()

In [19]:
filename = 'cell_count_layers_onlyPVasInhib.txt'

fullpath = filename
with open(fullpath, 'w') as outfile:
    json.dump(num_neurons_layers, outfile)

#### Only Pvalb as inhibitory:

In [27]:
firing_rate_layers = {}
firing_rate_layers['23e'] = f_rates_trial_avg[0].tolist()
firing_rate_layers['4e'] = np.sum(f_rates_trial_avg[1:5], axis=0).tolist()
firing_rate_layers['5e'] = np.sum(f_rates_trial_avg[5:7], axis=0).tolist()
firing_rate_layers['6e'] = f_rates_trial_avg[7].tolist()
firing_rate_layers['1i'] = f_rates_trial_avg[8].tolist()
firing_rate_layers['23i'] = f_rates_trial_avg[10].tolist()
firing_rate_layers['4i'] = f_rates_trial_avg[13].tolist()
firing_rate_layers['5i'] = f_rates_trial_avg[16].tolist()
firing_rate_layers['6i'] = f_rates_trial_avg[19].tolist()

### Save firing rates

In [28]:
filename = 'week3_2020/mod_21_jan_20/firing_rate_layerpops_onlyPVasInhib.txt'

In [29]:
fullpath = os.path.join(sim_data_dir, filename)
with open(fullpath, 'w') as outfile:
    json.dump(firing_rate_layers, outfile)